"""
# Hartree-Fock Analytic Nuclear Gradients Implementation

This notebook demonstrates how to compute analytic nuclear gradients for Hartree-Fock theory using Psi4Numpy.

## Theory Overview
For a closed-shell system, in the MO-basis, the Hartree-Fock energy gradient with respect to nuclear coordinate x is:

\begin{align}
\frac{dE_{\text{RHF}}}{dR_A} &=
\sum_{i} \left( \frac{\partial h_{ii}}{\partial R_A} + \frac{\partial G_{ii}}{\partial R_A} \right)
- \sum_{pq} F_{pq} \frac{\partial S_{pq}}{\partial R_A}
+ \frac{\partial E_{\text{nuc}}}{\partial R_A}
\end{align}

where

   $ i, j $ run over occupied orbitals

   $ a, b $ run over virtual orbitals

   $ p, q $ run over all orbitals

   $ h_{ii} $ is the one-electron integral in MO basis

   $ G_{ii} $ is the two-electron (Coulomb-exchange) contribution

   $ F_{pq} $ is the Fock matrix
   
   $ S_{pq} $ is the MO overlap matrix


In [ ]:
import psi4
import numpy as np
np.set_printoptions(precision=15, linewidth=200, suppress=True)

# Define your molecule
molecule = psi4.geometry("""
O 0.0 0.0 0.0
H 0.0 0.757 0.587
H 0.0 -0.757 0.587
symmetry c1
""")

# Set up a basis set
basis = psi4.core.BasisSet.build(molecule, target="sto-3g")

# Run an RHF calculation
psi4.set_options({'SCF_TYPE': 'PK', 'D_CONVERGENCE': 1.0e-8})
rhf_e, rhf_wfn = psi4.energy('SCF/sto-3g', molecule=molecule, return_wfn=True)





# Create a MintsHelper object
mints = psi4.core.MintsHelper(basis)

n_atoms = molecule.natom()
n_basis = basis.nbf()
ndocc = rhf_wfn.nalpha()

# Assuming C1 symmetry    

C = rhf_wfn.Ca_subset("AO", "ALL")
npC = np.asarray(C)

H_ao = np.asarray(mints.ao_kinetic()) + np.asarray(mints.ao_potential())

# Update H, transform to MO basis 
H = np.einsum('uj,vi,uv', npC, npC, H_ao)

# Integral generation from Psi4's MintsHelper
MO = np.asarray(mints.mo_eri(C, C, C, C))
# Physicist notation    
MO = MO.swapaxes(1,2)

# build Fock matrix
F = H + 2.0 * np.einsum('pmqm->pq', MO[:, :ndocc, :, :ndocc])
F -= np.einsum('pmmq->pq', MO[:, :ndocc, :ndocc, :])


nuclear_gradient = np.zeros((n_atoms, 3))
overlap_gradient = np.zeros((n_atoms, 3))
potential_gradient = np.zeros((n_atoms, 3))
kinetic_gradient = np.zeros((n_atoms, 3))
coulomb_gradient = np.zeros((n_atoms, 3))
exchange_gradient = np.zeros((n_atoms, 3))
total_gradient = np.zeros((n_atoms, 3))

nuclear_gradient = np.asarray(molecule.nuclear_repulsion_energy_deriv1())

print("Nuclear gradient:")
print(nuclear_gradient.reshape((n_atoms, 3)))


In [ ]:
# contract ERI_deriv to J_deriv and K_deriv
for atom_index in range(n_atoms):
    for cart_index in range(3):
        deriv_index = 3 * atom_index + cart_index
        tei_deriv = eri_derivs[deriv_index]
        J_deriv[deriv_index, :, :] += 2 * np.einsum("pqrs,rs->pq", tei_deriv, D)
        K_deriv[deriv_index, :, :] -= np.einsum("prqs,rs->pq", tei_deriv, D)
        

In [ ]:
# Build the Fock matrix at the converged geometry
fock_matrix = np.asarray(rhf_wfn.Fa())

# Build the energy-weighted density matrix
energy_weighted_density = W


electronic_gradient = np.zeros(3 * n_atoms)
kinetic_gradient = np.zeros(3 * n_atoms)
potential_gradient = np.zeros(3 * n_atoms)
overlap_gradient = np.zeros(3 * n_atoms)
coulomb_gradient = np.zeros(3 * n_atoms)
exchange_gradient = np.zeros(3 * n_atoms)


for atom_index in range(n_atoms):
    for cart_index in range(3):
        deriv_index = 3 * atom_index + cart_index

        # Contribution from one-electron integral derivatives
        h_deriv = kinetic_derivs[deriv_index] + potential_derivs[deriv_index]
        electronic_gradient[deriv_index] += np.sum(density_matrix * h_deriv)
        kinetic_gradient[deriv_index] += np.sum(density_matrix * kinetic_derivs[deriv_index])
        potential_gradient[deriv_index] += np.sum(density_matrix * potential_derivs[deriv_index])

        coulomb_gradient[deriv_index] += np.sum(D * J_deriv[deriv_index])
        exchange_gradient[deriv_index] += np.sum(D * K_deriv[deriv_index])

        # Contribution from two-electron integral derivatives
        #J = np.einsum("pqrs,rs->pq", I, D)
        #K = np.einsum("prqs,rs->pq", I, D)
        #tei_deriv = eri_derivs[deriv_index]
        #print("Printing TEI_DERIV Shape")
        #print(tei_deriv.shape)
        #print("Printing D shape")
        #print(D.shape)
        #coulomb_gradient[deriv_index] += 2 * np.einsum("pqrs,rs->pq", tei_deriv, D)
        #exchange_gradient[deriv_index] -= np.einsum("prqs,rs->pq", tei_deriv, D)
        #electronic_gradient[deriv_index] += 0.5 * np.sum(density_matrix[:, :, None, None] * density_matrix[None, None, :, :] * tei_deriv)
        #coulomb_plus_exchange_gradient[deriv_index] += 0.5 * np.sum(density_matrix[:, :, None, None] * density_matrix[None, None, :, :] * tei_deriv)
        electronic_gradient[deriv_index] += coulomb_gradient[deriv_index] + exchange_gradient[deriv_index]
        # Pulay contribution from overlap derivative
        overlap_deriv = overlap_derivs[deriv_index]
        electronic_gradient[deriv_index] -= np.sum(energy_weighted_density * overlap_deriv)
        overlap_gradient[deriv_index] -= np.sum(density_matrix * overlap_deriv)

print("Electronic Energy Gradient:\n", electronic_gradient.reshape(3,3))
print("Nuclear Energy Gradient:\n", nuclear_repulsion_gradients)
print("Total Gradient:\n", electronic_gradient.reshape(3,3) + nuclear_repulsion_gradients)

## From output of this script https://github.com/psi4/psi4/blob/master/samples/psi4numpy/rhf-gradient/input.py
## when the current H2O geometry is used
_expected_nuclear_gradient = np.array([
    [0.00000000000000,  0.00000000000000,  2.99204046891092],
    [0.00000000000000, -2.05144597283373, -1.49602023445546],
    [0.00000000000000,  2.05144597283373, -1.49602023445546],
])

_overlap_gradient = np.array([
    [-0.00000000000000, -0.00000000000000,  0.30728746121587],
    [ 0.00000000000000, -0.14977126575800, -0.15364373060793],
    [-0.00000000000000,  0.14977126575800, -0.15364373060793],
])

_potential_gradient = np.array([
    [-0.00000000000000,  0.00000000000002, -6.81982772856799],
    [-0.00000000000000,  4.38321774316664,  3.40991386428399],
    [ 0.00000000000000, -4.38321774316666,  3.40991386428400],
])

_kinetic_gradient = np.array([
    [ 0.00000000000000, -0.00000000000000,  0.66968290617933],
    [ 0.00000000000000, -0.43735698924315, -0.33484145308966],
    [-0.00000000000000,  0.43735698924315, -0.33484145308967],
])

_coulomb_gradient = np.array([
    [ 0.00000000000000, -0.00000000000002,  3.34742251141627],
    [ 0.00000000000000, -2.03756324433539, -1.67371125570813],
    [-0.00000000000000,  2.03756324433541, -1.67371125570814],
])

_exchange_gradient = np.array([
    [-0.00000000000000,  0.00000000000000, -0.43559674130726],
    [-0.00000000000000,  0.26932748463493,  0.21779837065363],
    [ 0.00000000000000, -0.26932748463493,  0.21779837065363],
])

# Compare the calculated gradients with the expected values
print("Nuclear Gradients Agree:\n")
print(np.allclose(nuclear_repulsion_gradients.reshape(3,3), _expected_nuclear_gradient))

print("Overlap Gradients Agree:\n")
print(np.allclose(overlap_gradient.reshape(3,3), _overlap_gradient))
print("Printing Overlap Gradient")
print(overlap_gradient.reshape(3,3))
print("Printing Expected Overlap Gradient")
print(_overlap_gradient)

print("Kinetic Gradients Agree:\n")
print(np.allclose(kinetic_gradient.reshape(3,3), _kinetic_gradient))

print("Potential Gradients Agree:\n")
print(np.allclose(potential_gradient.reshape(3,3), _potential_gradient))

print("Coulomb Gradients Agree:\n")
print(np.allclose(coulomb_gradient.reshape(3,3), _coulomb_gradient))
print("Printing Coulomb Gradient")
print(coulomb_gradient.reshape(3,3))
print("Printing Expected Coulomb Gradient")
print(_coulomb_gradient)

print("Exchange Gradients Agree:\n")
print(np.allclose(exchange_gradient.reshape(3,3), _exchange_gradient))
print("Printing Exchange Gradient")
print(exchange_gradient.reshape(3,3))
print("Printing Expected Exchange Gradient")
print(_exchange_gradient)

#_jk_gradient = _coulomb_gradient + _exchange_gradient

#print(coulomb_gradient.reshape(3,3) - _coulomb_gradient)
#print(exchange_gradient.reshape(3,3) - _exchange_gradient)